# Imports

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sp
from joblib import dump, load
from collections import namedtuple

%matplotlib inline
plt.style.use('seaborn')
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

from google.colab import drive
import os
%load_ext autoreload
%autoreload 2

drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Mounted at /content/gdrive


In [2]:
!pip install antspyx

     |████████████████████████████████| 301.6MB 21kB/s 
     |████████████████████████████████| 71kB 8.3MB/s 


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
!pip install deepbrain

     |████████████████████████████████| 686kB 5.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=80ceaa440319cfe2541ecbf9812335307e9e901910e92183f307e8edf031b58f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


# Preprocessing and Reprocessing MRIs Faster with multiple notebooks

In [27]:
os.chdir('/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/src/data_preprocessing/')
from mri_preprocessing import *

output_path = "/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210523/"
input_path='/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI'

In [28]:
images_to_process,_,_ = list_available_images(input_path)
images_to_process[:10]

Looking for MRI images in path: /content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI 

Found a total of  1202  images.
Found a total of  0  mask images.
Available images to process:  1202 



['/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/057_S_0464/MPR-R__GradWarp__B1_Correction__N3__Scaled/2009-06-24_13_49_46.0/S69136/ADNI_057_S_0464_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20091203185358387_S69136_I160245.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/036_S_0673/MPR__GradWarp__B1_Correction__N3__Scaled/2006-07-21_08_45_17.0/S17157/ADNI_036_S_0673_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070120001203137_S17157_I36949.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/033_S_0514/MPR__GradWarp__B1_Correction__N3__Scaled/2007-06-07_15_43_00.0/S33320/ADNI_033_S_0514_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070809185451369_S33320_I66544.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/013_S_0502/MPR__GradWarp__B1_Correction__N3__Scal

In [30]:
raw_image_names = [x.split(sep='/')[-1] for x in images_to_process]
processed_images,_,_ = list_available_images(output_path,file_format='.nii.gz')
processed_image_names = [x.split(sep='/20210523/')[-1].split('.gz')[0] for x in processed_images]
image_names_to_reprocess = list(set(raw_image_names) - set(processed_image_names))
images_to_reprocess = [x for x in images_to_process if x.split(sep='/')[-1] in image_names_to_reprocess]
len(images_to_reprocess)

Looking for MRI images in path: /content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/preprocessed/20210523/ 

Found a total of  800  images.
Found a total of  0  mask images.
Available images to process:  800 



1202

## Split image array

In [31]:
images_to_reprocess[:300]

['/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/057_S_0464/MPR-R__GradWarp__B1_Correction__N3__Scaled/2009-06-24_13_49_46.0/S69136/ADNI_057_S_0464_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20091203185358387_S69136_I160245.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/036_S_0673/MPR__GradWarp__B1_Correction__N3__Scaled/2006-07-21_08_45_17.0/S17157/ADNI_036_S_0673_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070120001203137_S17157_I36949.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/033_S_0514/MPR__GradWarp__B1_Correction__N3__Scaled/2007-06-07_15_43_00.0/S33320/ADNI_033_S_0514_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070809185451369_S33320_I66544.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/013_S_0502/MPR__GradWarp__B1_Correction__N3__Scal

In [32]:
images_to_reprocess[300:600]

['/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/141_S_0790/MPR__GradWarp__B1_Correction__N3__Scaled/2007-12-04_16_13_52.0/S47536/ADNI_141_S_0790_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080410151355187_S47536_I102146.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/128_S_0522/MPR__GradWarp__B1_Correction__N3__Scaled/2006-12-15_16_45_14.0/S24132/ADNI_128_S_0522_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070819171015724_S24132_I68991.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/100_S_0930/MPR____N3__Scaled/2007-04-23_10_59_47.0/S30198/ADNI_100_S_0930_MR_MPR____N3__Scaled_Br_20070911121409167_S30198_I72526.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/114_S_0458/MPR__GradWarp__B1_Correction__N3__Scaled/2006-11-27_13_01_39.0/S22982/ADNI_114_S_0458_MR

In [33]:
images_to_reprocess[600:900]

['/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/009_S_0751/MPR__GradWarp__N3__Scaled/2006-07-25_12_31_19.0/S17487/ADNI_009_S_0751_MR_MPR__GradWarp__N3__Scaled_Br_20080408151939286_S17487_I101528.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/027_S_0644/MPR__GradWarp__B1_Correction__N3__Scaled/2007-06-29_14_40_12.0/S34297/ADNI_027_S_0644_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070816141329245_S34297_I68088.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/032_S_0677/MPR____N3__Scaled_2/2007-07-19_11_04_37.0/S35508/ADNI_032_S_0677_MR_MPR____N3__Scaled_2_Br_20081003103356239_S35508_I119104.nii',
 '/content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/010_S_0786/MPR____N3__Scaled/2007-10-10_10_36_59.0/S41161/ADNI_010_S_0786_MR_MPR____N3__Scaled_Br_20071027134932531_S41161_I

In [ ]:
images_to_reprocess[900:]

## Execute

In [ ]:
execute_preprocessing(output_path=output_path, 
                      images_to_process = images_to_reprocess[:300],
                      box=100)

Setting ANTs and NiftyReg environment variables...

------------------------------------------------------------------------------------------------------------------------
Starting pre-processing (Labeling + Standardizing + Registration + Skull Stripping + Cropping) for 300 images. This might take a while... =)
Processing from  image 0 to image 0.
------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------
Processing image (1/300): /content/gdrive/MyDrive/Lucas_Thimoteo/mmml-alzheimer-diagnosis/data/mri/raw/ADNI_01_02_1200/ADNI/057_S_0464/MPR-R__GradWarp__B1_Correction__N3__Scaled/2009-06-24_13_49_46.0/S69136/ADNI_057_S_0464_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20091203185358387_S69136_I160245.nii
Standardizing image based on Atlas...
Found a total of 0 NaN values in image.
Registering image to Atl